In [43]:
## caged_brach
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

import basedosdados as bd

from io import StringIO
import requests

In [23]:
def _sheet_to_df(columns_config_url):
    url = columns_config_url.replace("edit#gid=", "export?format=csv&gid=")

    return pd.read_csv(StringIO(requests.get(url).content.decode("utf-8")))
    
def normalize_cols(df):
    return (
        df.str.normalize("NFKD")
        .str.encode("ascii", errors="ignore")
        .str.decode("utf-8")
        .str.title()

    )

In [31]:
df = _sheet_to_df("https://docs.google.com/spreadsheets/d/1kfmQCZtRqc4msId0mtCwXbgAmubLm4SS_ReqMLm4Gz0/edit#gid=439989260")

In [32]:
df.columns

Index(['ano', 'DEPUTADO (A)', 'RECEBEDOR', 'BENEFICIÁRIO', 'LOCAL', 'CNPJ',
       'FONTE DOS RECURSOS', 'OBJETO', 'ESPECIFICAÇÃO', 'ORIGEM', 'TEMA',
       'ÁREA', 'VALOR', 'LATLON', 'FINALIDADE', 'FINALIDADE_WRAP'],
      dtype='object')

In [33]:
mask = df['ÁREA'] == 'SAÚDE'

cols = ['ÁREA','LOCAL','BENEFICIÁRIO','VALOR']
df = df[mask][cols]
df['nome'] = normalize_cols(df['LOCAL'])

In [26]:
query = """
SELECT
  t1.id_municipio,
  t1.nome,
  t1.mesorregiao,
  t2.populacao 
FROM `basedosdados.br_bd_diretorios_brasil.municipio` t1
JOIN `basedosdados.br_ibge_populacao.municipio` t2 
  ON t1.id_municipio = t2.id_municipio 
WHERE t1.sigla_uf='SP' 
  AND (t1.mesorregiao = 'Vale do Paraíba Paulista' or t1.id_municipio='3530607')
  AND t2.ano=2020
"""

pop = bd.read_sql(query, billing_project_id='basedosdados-dev')

Downloading: 100%|███████████████████████████████████████████████████| 40/40 [00:00<00:00, 105.89rows/s]


In [29]:
pop['nome'] = normalize_cols(pop['nome'])

In [36]:
dd = df.merge(pop, how='left', on='nome')

In [40]:
dd.drop_duplicates(subset=['id_municipio'])['populacao'].sum()

2117388

In [42]:
dd.drop_duplicates(subset=['id_municipio']).shape

(21, 8)

In [35]:
df.shape

(30, 5)